<a href="https://colab.research.google.com/github/ExCaLBBR/ExCaLBBR_Projects/blob/main/GeometryBasedDecisions/code/Gorilla2BIDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gorilla Info: <br>
Project: GeometryBasedDecisions <br>
Account: Roberto Vargas (robertov@andrew.cmu.edu)

In [1]:
#@title Install Dependancies
import pandas as pd
import numpy as np
import re

In [2]:
#@title Utility Functions

def convert_to_float(x):
    if isinstance(x, list):
        return float(x[0]) # assuming there is only one value in the list
    else:
        return x

In [3]:
#@title Load Data

#QUESTIONAIRRES:
#Demographics
url = 'https://raw.githubusercontent.com/ExCaLBBR/ExCaLBBR_Projects/refs/heads/main/GeometryBasedDecisions/data/raw/raw_demographic.csv'
df_demographic = pd.read_csv(url)

#News Consumption
url = 'https://raw.githubusercontent.com/ExCaLBBR/ExCaLBBR_Projects/refs/heads/main/GeometryBasedDecisions/data/raw/raw_questionnaire_NewsConsumption.csv'
df_News = pd.read_csv(url)

#Concept Engagement Pt1
url = 'https://github.com/ExCaLBBR/ExCaLBBR_Projects/raw/refs/heads/main/GeometryBasedDecisions/data/raw/raw_task_SocConEngagePt1.csv'
df_ConEngPt1 = pd.read_csv(url)

#Concept Engagement Pt23
url = 'https://github.com/ExCaLBBR/ExCaLBBR_Projects/raw/refs/heads/main/GeometryBasedDecisions/data/raw/raw_questionnaire_SocConEngagePt23.csv'
df_ConEngPt23 = pd.read_csv(url)

#PurposeQ
url = 'https://raw.githubusercontent.com/ExCaLBBR/ExCaLBBR_Projects/refs/heads/main/GeometryBasedDecisions/data/raw/raw_questionnaire_PurposeQ.csv'
df_PurposeQ = pd.read_csv(url)

#TASKS:
#PRaM
url = 'https://github.com/ExCaLBBR/ExCaLBBR_Projects/raw/refs/heads/main/GeometryBasedDecisions/data/raw/raw_task_PRaM.csv.gz'
df_taskPRaM = pd.read_csv(url, compression='gzip')


<ipython-input-3-8429c8b7a8a7>:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_demographic = pd.read_csv(url)
<ipython-input-3-8429c8b7a8a7>:14: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ConEngPt1 = pd.read_csv(url)
<ipython-input-3-8429c8b7a8a7>:27: DtypeWarning: Columns (0,36,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df_taskPRaM = pd.read_csv(url, compression='gzip')


In [4]:
#@title Variable column isolation
#Isolate relevant columns: Demographics
include = ['Participant Private ID', 'Question', 'Key', 'Response']
df_demographic.drop(columns=df_demographic.columns.difference(include), inplace=True)
mask = df_demographic['Response'].str.contains('BEGIN', na=True) | df_demographic['Response'].str.contains('END', na=True)
df_demographic = df_demographic[~mask]

#Isolate relevant columns: News Consumption
include = ['Participant Private ID', 'Question', 'Key', 'Response']
df_News.drop(columns=df_News.columns.difference(include), inplace=True)
mask = df_News['Response'].str.contains('BEGIN', na=True) | df_News['Response'].str.contains('END', na=True)
df_News = df_News[~mask]

#Isolate relevant columns: Concept Engagement Pt1
include = ['Participant Private ID', 'Spreadsheet: display', 'Object Name', 'Spreadsheet: Instituion', 'Spreadsheet: Question']
df_ConEngPt1.drop(columns=df_ConEngPt1.columns.difference(include), inplace=True)
mask = df_ConEngPt1['Spreadsheet: display'].str.contains('Trial', na=True)
df_ConEngPt1 = df_ConEngPt1[mask]
df_ConEngPt1 = df_ConEngPt1.dropna()

#Isolate relevant columns: Concept Engagement Pt23
include = ['Participant Private ID', 'Key', 'Response', 'Object ID']
df_ConEngPt23.drop(columns=df_ConEngPt23.columns.difference(include), inplace=True)
mask = df_ConEngPt23['Key'].str.contains('value', na=True)
df_ConEngPt23 = df_ConEngPt23[mask]
df_ConEngPt23 = df_ConEngPt23.dropna()

#Isolate relevant columns: PurposeQ
include = ['Participant Private ID', 'Question Key', 'Response']
df_PurposeQ.drop(columns=df_PurposeQ.columns.difference(include), inplace=True)
mask = df_PurposeQ['Question Key'].str.contains('BEGIN QUESTIONNAIRE', na=True) | df_PurposeQ['Question Key'].str.contains('END QUESTIONNAIRE', na=True) | df_PurposeQ['Question Key'].str.contains('OpenEnded', na=True)
df_PurposeQ = df_PurposeQ[~mask]

#Isolate relevant columns: PRaM
include = ['Participant Private ID', 'Zone Name', 'conceptA', 'conceptB', 'Response']
df_taskPRaM.drop(columns=df_taskPRaM.columns.difference(include), inplace=True)
mask = df_taskPRaM['Zone Name'].str.contains('Likert', na=True)
df_taskPRaM = df_taskPRaM[mask]
df_taskPRaM = df_taskPRaM.dropna()
df_taskPRaM.drop(columns=['Zone Name'], inplace=True)
df_taskPRaM['Pair'] = df_taskPRaM['conceptA'] + '-' + df_taskPRaM['conceptB']
df_taskPRaM.drop(columns=['conceptA', 'conceptB'], inplace=True)


In [5]:
#@title Convert to Pivot tables
#Isolate Pivot dataframes
df_demographic = df_demographic.pivot(index = 'Participant Private ID', columns = ['Question', 'Key'], values = ['Response']) #Demographics

df_News = df_News.pivot(index = 'Participant Private ID', columns = ['Question', 'Key'], values = ['Response']) #News Consumption

df_ConEngPt1 = df_ConEngPt1.pivot(index = 'Participant Private ID', columns = ['Spreadsheet: Instituion', 'Spreadsheet: Question'], values = ['Object Name'])

df_ConEngPt23 = df_ConEngPt23.pivot(index = 'Participant Private ID', columns = ['Object ID'], values = ['Response'])

df_PurposeQ = df_PurposeQ.pivot(index = 'Participant Private ID', columns = ['Question Key'], values = ['Response'])

#TASKS:
df_taskPRaM = df_taskPRaM.groupby(['Participant Private ID', 'Pair'])['Response'].agg(list).reset_index()
df_taskPRaM = df_taskPRaM.pivot(index='Participant Private ID', columns='Pair', values='Response')
df_taskPRaM = df_taskPRaM.applymap(convert_to_float)


<ipython-input-5-ab824a15709e>:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_taskPRaM = df_taskPRaM.applymap(convert_to_float)


In [6]:
#Convert all files to csv
df_demographic.to_csv('df_demographic.csv')
df_News.to_csv('df_News.csv')
df_ConEngPt1.to_csv('df_ConEngPt1.csv')
df_ConEngPt23.to_csv('df_ConEngPt23.csv')
df_PurposeQ.to_csv('df_PurposeQ.csv')
df_taskPRaM.to_csv('df_taskPRaM.csv')

In [ ]:
#Offline Processing is as follows:

#Remove nested columns
#Rename column headers to be more intuitive
#News: Added AllSides coding (10/23)